<a href="https://colab.research.google.com/github/LukeBirkett/football_analytics/blob/main/Building_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StatsBomb Arsenal Invincibles Season 03/04 Database Setup

In this project I am going to be taking the event data found in the [statsbomb/open-data](https://https://github.com/statsbomb/open-data) repository and attempt to set it up within my own database. The raw data is supplied in JSON files so this project will involve pulling in the data, unpacking it from the JSON format, cleaning it and pushing it into a database. 

I'll first start by reading in the packages I want to use and mounting my project to my google drive

In [36]:
import pandas as pd
import numpy as np
import json
import requests # not used

import sqlite3
import sqlalchemy

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#1. Competition ID's

First I am going to read in the JSON containing all of the competition details. I just want data for the **Premier League 03/04 season** which comes under `competition_id = 2` and `season_id = 44`

In [38]:
pd.read_json ('https://raw.githubusercontent.com/LukeBirkett/open-data/master/data/competitions.json')

,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,16,4,Europe,Champions League,male,2018/2019,2021-05-19T08:38:06.515138,2021-05-19T08:38:06.515138
1,16,1,Europe,Champions League,male,2017/2018,2021-01-23T21:55:30.425330,2021-01-23T21:55:30.425330
2,16,2,Europe,Champions League,male,2016/2017,2020-08-26T12:33:15.869622,2020-07-29T05:00
3,16,27,Europe,Champions League,male,2015/2016,2020-08-26T12:33:15.869622,2020-07-29T05:00
4,16,26,Europe,Champions League,male,2014/2015,2020-08-26T12:33:15.869622,2020-07-29T05:00
5,16,25,Europe,Champions League,male,2013/2014,2020-08-26T12:33:15.869622,2020-07-29T05:00
6,16,24,Europe,Champions League,male,2012/2013,2020-08-26T12:33:15.869622,2020-07-29T05:00
7,16,23,Europe,Champions League,male,2011/2012,2020-08-26T12:33:15.869622,2020-07-29T05:00
8,16,22,Europe,Champions League,male,2010/2011,2020-07-29T05:00,2020-07-29T05:00
9,16,21,Europe,Champions League,male,2009/2010,2020-07-29T05:00,2020-07-29T05:00


# 2. Match ID's

Next we want to pull in the JSON file that holds information on the matches that took place. Using the `competition_id` and `season_id` we can manipulate the raw Github link to point to the competitions/seasons we want. In this case we are only looking at one season but if there were multiple we could use a loop to iterate through the ID's

We only really want to the Match ID's from this datasets but I will clean up the data so it is easier to look at

In [39]:
comp = "2"
season = "44"
url = 'https://raw.githubusercontent.com/LukeBirkett/open-data/master/data/matches/{}/{}.json'.format(comp, season)

matches = pd.read_json (url)

matches[['competition_id','country_name', 'competition_name']] = pd.DataFrame(matches.competition.values.tolist(), index= matches.index)
matches[['season_id','season_name']] = pd.DataFrame(matches.season.values.tolist(), index= matches.index)
matches[['home_team_id','home_team_name','home_team_gender','home_team_group','country','home_managers']] = pd.DataFrame(matches.home_team.values.tolist(), index= matches.index)
matches[['away_team_id','away_team_name','away_team_gender','away_team_group','country','away_managers']] = pd.DataFrame(matches.away_team.values.tolist(), index= matches.index)
matches = matches[["match_id","match_date","kick_off","home_score","away_score","match_status","match_week","competition_id","competition_name","season_id","season_name","home_team_name","away_team_name","home_managers","away_managers"]]
matches

,match_id,match_date,kick_off,home_score,away_score,match_status,match_week,competition_id,competition_name,season_id,season_name,home_team_name,away_team_name,home_managers,away_managers
0,3749257,2004-05-15,16:00:00.000,2,1,available,38,2,Premier League,44,2003/2004,Arsenal,Leicester City,"[{'id': 577, 'name': 'Arsène Wenger', 'nicknam...","[{'id': 2974, 'name': 'Micky Adams', 'nickname..."
1,3749246,2004-03-28,17:05:00.000,1,1,available,30,2,Premier League,44,2003/2004,Arsenal,Manchester United,NaN,NaN
2,3749153,2004-01-10,16:00:00.000,4,1,available,21,2,Premier League,44,2003/2004,Arsenal,Middlesbrough,"[{'id': 577, 'name': 'Arsène Wenger', 'nicknam...","[{'id': 40, 'name': 'Steve McClaren', 'nicknam..."
3,3749642,2004-02-28,16:00:00.000,2,1,available,27,2,Premier League,44,2003/2004,Arsenal,Charlton Athletic,NaN,NaN
4,3749358,2003-08-24,17:05:00.000,0,4,available,2,2,Premier League,44,2003/2004,Middlesbrough,Arsenal,NaN,NaN
5,3749346,2003-11-01,16:00:00.000,1,4,available,11,2,Premier League,44,2003/2004,Leeds United,Arsenal,NaN,NaN
6,3749253,2003-10-04,13:30:00.000,1,2,available,8,2,Premier League,44,2003/2004,Liverpool,Arsenal,NaN,NaN
7,3749079,2003-10-18,16:00:00.000,2,1,available,9,2,Premier League,44,2003/2004,Arsenal,Chelsea,NaN,NaN
8,3749465,2004-03-13,16:00:00.000,0,2,available,28,2,Premier League,44,2003/2004,Blackburn Rovers,Arsenal,NaN,NaN
9,3749133,2004-01-18,15:00:00.000,0,2,available,22,2,Premier League,44,2003/2004,Aston Villa,Arsenal,NaN,NaN


In [40]:
for x in matches["match_id"]:
 print(x)

3749257
3749246
3749153
3749642
3749358
3749346
3749253
3749079
3749465
3749133
3749528
3749233
3749462
3749552
3749296
3749454
3749276
3749068
3749310
3749493
3749434
3749192
3749196
3749522
3749448
3749403
3749360
3749453
3749278
3749526
3749052
3749603
3749431


#3. Reading in Match Events

Now have the match ID's we can read in the events data. 

##3.1. Single Match Test
Before reading in all off the matches I am going to test run on one match. I can see that there are field names of type and pass which I know are resevered phrases in Python so I will rename these to avoid bugs. Also I can see that is not as simple as just reading in the overal JSON as nom columns can hold further information nested within a `dictionary` format

In [41]:
events = pd.read_json ('https://raw.githubusercontent.com/LukeBirkett/open-data/master/data/events/3749257.json')
events = events.rename(columns={"type":"type_name", "pass": "pass_details"})
events

,id,index,period,timestamp,minute,second,type_name,possession,possession_team,play_pattern,team,duration,tactics,related_events,player,position,location,pass_details,carry,under_pressure,clearance,counterpress,duel,ball_receipt,out,off_camera,dribble,shot,goalkeeper,ball_recovery,foul_committed,foul_won,interception,miscontrol,block,substitution
0,41e0ff39-da7c-451a-8f08-82d3a9b369f2,1,1,2021-09-15 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 1, 'name': 'Arsenal'}",0.000000,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d8c32d32-494b-4ae1-bb0c-d2f738952e3c,2,1,2021-09-15 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 22, 'name': 'Leicester City'}",0.000000,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6e678cba-67c3-4e9a-acca-78ab69b7d68b,3,1,2021-09-15 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 1, 'name': 'Arsenal'}",0.000000,NaN,[b31e69b0-a75e-4721-b023-c06094ddcfa0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b31e69b0-a75e-4721-b023-c06094ddcfa0,4,1,2021-09-15 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 22, 'name': 'Leicester City'}",0.000000,NaN,[6e678cba-67c3-4e9a-acca-78ab69b7d68b],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0613063a-1cd4-4a18-83a7-9722be2d9f40,5,1,2021-09-15 00:00:01.036,0,1,"{'id': 30, 'name': 'Pass'}",2,"{'id': 22, 'name': 'Leicester City'}","{'id': 9, 'name': 'From Kick Off'}","{'id': 22, 'name': 'Leicester City'}",0.238292,NaN,[500a6fd9-61c7-4b61-bb3c-8e6605c24084],"{'id': 40240, 'name': 'Paul Dickov'}","{'id': 24, 'name': 'Left Center Forward'}","[61.0, 40.1]","{'recipient': {'id': 40242, 'name': 'Marcus Be...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,9ec50d3f-de8a-414d-9bcd-c8592f84fd35,3452,2,2021-09-15 00:46:41.121,91,41,"{'id': 42, 'name': 'Ball Receipt*'}",185,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 1, 'name': 'Arsenal'}",NaN,NaN,[d7d1ac66-3c07-4038-b05f-e1aa60c47d46],"{'id': 15654, 'name': 'Martin Keown'}","{'id': 12, 'name': 'Right Midfield'}","[68.5, 75.2]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3452,f4a899d4-d320-4ddd-8401-0feb57e61d9c,3453,2,2021-09-15 00:46:41.121,91,41,"{'id': 4, 'name': 'Duel'}",185,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 22, 'name': 'Leicester City'}",0.000000,NaN,[9ce7f4cf-2e65-4708-9c01-724379295d09],"{'id': 40239, 'name': 'Jordan Stewart'}","{'id': 6, 'name': 'Left Back'}","[51.6, 4.9]",NaN,NaN,1.0,NaN,NaN,"{'type': {'id': 10, 'name': 'Aerial Lost'}}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3453,9ce7f4cf-2e65-4708-9c01-724379295d09,3454,2,2021-09-15 00:46:41.121,91,41,"{'id': 38, 'name': 'Miscontrol'}",185,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 1, 'name': 'Arsenal'}",0.000000,NaN,[f4a899d4-d320-4ddd-8401-0feb57e61d9c],"{'id': 15654, 'name': 'Martin Keown'}","{'id': 12, 'name': 'Right Midfield'}","[68.5, 75.2]",NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'aerial_won': True},NaN,NaN
3454,44c9a058-df2b-4f2b-bc66-8f8d26a90060,3455,2,2021-09-15 00:46:47.136,91,47,"{'id': 34, 'name': 'Half End'}",185,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 1, 'name': 'Arsenal'}",0.00

##3.2. All Matches Dataframe

Now I understand what the data will look like I can just read in all of data into one larger dataset. To do this I first create a function which will read in individual matches add create a match_id column then I use this function within a for loop to iterate through all of the matches which I finally merge together

In [42]:
def read_match_events(match_id):
  df = pd.read_json(f'https://raw.githubusercontent.com/LukeBirkett/open-data/master/data/events/{match_id}.json')
  df['match_id'] = match_id
  return df

#option to create a subset for each game
#for x in matches["match_id"]:
#  globals()['match_%s' %x] = read_match_events(x)

list_matches_df = [] # store each unpacked match in this list
for x in matches["match_id"]:
  data = read_match_events(x)
  list_matches_df.append(data)

matchesDF = pd.concat(list_matches_df, axis=0) # merge together the list
matchesDF = matchesDF.rename(columns={"type":"type_name", "pass": "pass_details"})
matchesDF = matchesDF.drop(columns=['50_50']) #dropped as a quick fix to avoid bugs
display(matchesDF)

,id,index,period,timestamp,minute,second,type_name,possession,possession_team,play_pattern,team,duration,tactics,related_events,player,position,location,pass_details,carry,under_pressure,clearance,counterpress,duel,ball_receipt,out,off_camera,dribble,shot,goalkeeper,ball_recovery,foul_committed,foul_won,interception,miscontrol,block,substitution,match_id,bad_behaviour,half_start,injury_stoppage
0,41e0ff39-da7c-451a-8f08-82d3a9b369f2,1,1,2021-09-15 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 1, 'name': 'Arsenal'}",0.000000,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3749257,NaN,NaN,NaN
1,d8c32d32-494b-4ae1-bb0c-d2f738952e3c,2,1,2021-09-15 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 22, 'name': 'Leicester City'}",0.000000,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3749257,NaN,NaN,NaN
2,6e678cba-67c3-4e9a-acca-78ab69b7d68b,3,1,2021-09-15 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 1, 'name': 'Arsenal'}",0.000000,NaN,[b31e69b0-a75e-4721-b023-c06094ddcfa0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3749257,NaN,NaN,NaN
3,b31e69b0-a75e-4721-b023-c06094ddcfa0,4,1,2021-09-15 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 1, 'name': 'Arsenal'}","{'id': 1, 'name': 'Regular Play'}","{'id': 22, 'name': 'Leicester City'}",0.000000,NaN,[6e678cba-67c3-4e9a-acca-78ab69b7d68b],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3749257,NaN,NaN,NaN
4,0613063a-1cd4-4a18-83a7-9722be2d9f40,5,1,2021-09-15 00:00:01.036,0,1,"{'id': 30, 'name': 'Pass'}",2,"{'id': 22, 'name': 'Leicester City'}","{'id': 9, 'name': 'From Kick Off'}","{'id': 22, 'name': 'Leicester City'}",0.238292,NaN,[500a6fd9-61c7-4b61-bb3c-8e6605c24084],"{'id': 40240, 'name': 'Paul Dickov'}","{'id': 24, 'name': 'Left Center Forward'}","[61.0, 40.1]","{'recipient': {'id': 40242, 'name': 'Marcus Be...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3749257,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,85737fb5-4776-4953-bb11-dccbe4d21ec7,3244,2,2021-09-15 00:48:06.607,93,6,"{'id': 43, 'name': 'Carry'}",204,"{'id': 37, 'name': 'Newcastle United'}","{'id': 7, 'name': 'From Goal Kick'}","{'id': 1, 'name': 'Arsenal'}",0.415176,NaN,"[09efb9b4-b735-49ad-890b-0a7583496c00, 8f58481...","{'id': 20302, 'name': 'Gaël Clichy'}","{'id': 6, 'name': 'Left Back'}","[40.7, 31.1]",NaN,"{'end_location': [41.6, 31.1]}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3749431,NaN,NaN,NaN
3244,8f58481a-7d84-4db9-a4fa-97365676c2d2,3245,2,2021-09-15 00:48:07.022,93,7,"{'id': 30, 'name': 'Pass'}",204,"{'id': 37, 'name': 'Newcastle United'}","{'id': 7, 'name': 'From Goal Kick'}","{'id': 1, 'name': 'Arsenal'}",1.389229,NaN,[c9f20a6b-13fb-4698-966c-ea2e16cbd42b],"{'id': 20302, 'name': 'Gaël Clichy'}","{'id': 6, 'name': 'Left Back'}","[41.6, 31.1]","{'recipient': {'id': 26014, 'name': 'Eduardo C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3749431,NaN,NaN,NaN
3245,c9f20a6b-13fb-4698-966c-ea2e16cbd42b,3246,2,2021-09-15 00:48:08.411,93,8,"{'id': 42, 'name': 'Ball Receipt*'}",204,"{'id': 37, 'name': 'Newcastle United'}","{'id': 7, 'name': 'From Goal Kick'}","{'id': 1, 'name': 'Arsenal'}",NaN,NaN,[8f58481a-7d84-4db9-a4fa-97365676c2d2],"{'id': 26014, 'name': 'Eduardo César Daude Gas...","{'id': 16, 'name': 'Left Midfield'}","[49.9, 21.4]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## 3.3. Turning Fields into Subsets

Given every event as an unique ID I am going to break each field up into its own subset from which it can be joined back to the UID later. This will make cleaning the data much easier. Also I will not need all the fields/categories for the analysis I want to do so I can just format the data I need for now. To do this I am simply looping through the dataset and setting up new subsets which hold two colums: id and the category title

In [43]:
for i in list(matchesDF):
  exec("{} = matchesDF.query(\"{} == {}\")[[\"id\",\"{}\"]]".format(i,i,i,i))

# example: pass_details = events.query("pass_details == pass_details")[["id","pass_details"]]

In [ ]:
print(list(matchesDF))

# pass_details team location player shot

pass_details

## 3.4. Unpack Each Subset, First Iteration
The general idea is that now each subset can be developed individually and speced out as desesired. However, I know by look at the data that the first step for the majority of most datasets is unesting the original dictionary so I will set up a loop to tackle this

In [45]:
for i in list(matchesDF)[1:]:
  exec("listNames = list(pd.DataFrame({}.{}.tolist()))".format(i,i))
  # example: list(pd.DataFrame(pass_details.pass_details.tolist()))
  # output: ['recipient','length','angle','height','end_location','body_part','type','outcome','switch','cross','assisted_shot_id','shot_assist','aerial_won','deflected','inswinging','technique','outswinging','goal_assist','through_ball','no_touch','miscommunication','straight','cut_back']

  listNames_rename_ = ['type_' if z=='type' else z for z in listNames]
  listNames_rename = ['id_' if z=='id' else z for z in listNames_rename_] #avoid field id overwritting event id
  
  if len(listNames) == 1:
    # if the length is 1 then it doesn't have any dictionary to unested
    continue

  exec("{}[listNames_rename] = pd.DataFrame({}.{}.values.tolist(), index= {}.index)".format(i,i,i,i))
  # example: pass_details[listNames_rename] = pd.DataFrame(pass_details.pass_details.values.tolist(), index= pass_details.index)
 
  exec("{} = {}.drop(columns=['{}'])".format(i,i,i))
  # example: pass_details = pass_details.drop(columns=['pass_details'])

#4. Planning Schemas
Now this is not to say all of our data is now unested and ready to use. A lot of the new columns created hold further nested information, particular our complex/detailed fields such as pass_detail and shot. However, this does give us a good base to start building up our datasets and decide how to organise them

Below I will outline the 4 states of development: 

##4.1. Base Fields
Some of these catgories do not need any further processing so I will pull these out from events a treat this as my base dataset to which information can be joined to once I get to the analysis part. Also these fields are mostly general fields which provide useful context for any analysis

In [46]:
base_events = matchesDF[['id','match_id','index','period','timestamp','minute','second','possession','duration','related_events']]

base_events = base_events.rename(columns={"index":"index_"})

base_events

,id,match_id,index_,period,timestamp,minute,second,possession,duration,related_events
0,41e0ff39-da7c-451a-8f08-82d3a9b369f2,3749257,1,1,2021-09-15 00:00:00.000,0,0,1,0.000000,NaN
1,d8c32d32-494b-4ae1-bb0c-d2f738952e3c,3749257,2,1,2021-09-15 00:00:00.000,0,0,1,0.000000,NaN
2,6e678cba-67c3-4e9a-acca-78ab69b7d68b,3749257,3,1,2021-09-15 00:00:00.000,0,0,1,0.000000,[b31e69b0-a75e-4721-b023-c06094ddcfa0]
3,b31e69b0-a75e-4721-b023-c06094ddcfa0,3749257,4,1,2021-09-15 00:00:00.000,0,0,1,0.000000,[6e678cba-67c3-4e9a-acca-78ab69b7d68b]
4,0613063a-1cd4-4a18-83a7-9722be2d9f40,3749257,5,1,2021-09-15 00:00:01.036,0,1,2,0.238292,[500a6fd9-61c7-4b61-bb3c-8e6605c24084]
...,...,...,...,...,...,...,...,...,...,...
3243,85737fb5-4776-4953-bb11-dccbe4d21ec7,3749431,3244,2,2021-09-15 00:48:06.607,93,6,204,0.415176,"[09efb9b4-b735-49ad-890b-0a7583496c00, 8f58481..."
3244,8f58481a-7d84-4db9-a4fa-97365676c2d2,3749431,3245,2,2021-09-15 00:48:07.022,93,7,204,1.389229,[c9f20a6b-13fb-4698-966c-ea2e16cbd42b]
3245,c9f20a6b-13fb-4698-966c-ea2e16cbd42b,3749431,3246,2,2021-09-15 00:48:08.411,93,8,204,NaN,[8f58481a-7d84-4db9-a4fa-97365676c2d2]
3246,c81c1693-282e-4d6d-9fa9-1622d35eab03,3749431,3247,2,2021-09-15 00:48:11.489,93,11,204,0.000000,[a14a6a86-9346-4c5f-bf0f-6fa9fed29352]


##4.2. Single Field Categories
Fields that don't require any further  unpacking

In [47]:
possession_team = possession_team.rename(columns={"id_":"possesion_team_id", "name":"possesion_team_name"})

play_pattern = play_pattern.rename(columns={"id_":"play_pattern_id", "name":"play_pattern_name"})

team = team.rename(columns={"id_":"team_id", "name":"team_name"})

player = player.rename(columns={"id_":"player_id", "name":"player_name"})

position = position.rename(columns={"id_":"position_id", "name":"position_name"})

counterpress

under_pressure

block

foul_won

out

,id,out
52,7e5bf8c9-a83f-45a5-a565-08acc081f271,1.0
442,fcf38da9-5183-4104-adba-756b3a1f5053,1.0
472,07666060-7460-4615-a9f1-e44bc9b9b8fc,1.0
743,3ce2c754-60d6-46de-9431-8c1ed06e65fe,1.0
793,0b5ab6af-46fb-4951-9519-3ad563a73ebc,1.0
...,...,...
2834,a3ead85f-4782-48db-8801-d89dde7c2102,1.0
2852,567eba9a-7cb6-4b5d-aa6f-526ecab3799d,1.0
2947,7141c5c6-8194-459b-b3b9-39cb17d4069b,1.0
3167,b70cf78d-f62a-41a0-9635-0a3dc668d984,1.0


##4.3. First To Be Developed
This are the subsets which I think will be most useful to the projects that I want to work on first

In [48]:
# location
# pass_details
# shot
# carry
# ball_recovery
# duel
# dribble
# interception
# ball_receipt

##4.4. Develop Another Time
These are the subsets which are less important to me right now. It would be a better use of my time to develop the overall process first before putting these subsets in

In [49]:
# tactics
# location
# goalkeeper
# foul_committed
# foul_won
# bad_behaviour
# substitution
# clearance
# off_camera
# miscontrol
# half_start
# injury_stoppage

#5. Developing Schemas

##5.1. location
Location is arguably the most important field in the event data. It tells us the location on the pitch that things look place

In [50]:
location
location = location.rename(columns={0: "x", 1: "y"})

#location['y'] = 80 - location['y'] # statsbomb has historically had their y coords flipped but this doesn't seem to be the case anymore

#location.x.min()
#location.x.max()
#location.y.min()
#location.y.max()

location

,id,x,y
4,0613063a-1cd4-4a18-83a7-9722be2d9f40,61.0,40.1
5,500a6fd9-61c7-4b61-bb3c-8e6605c24084,60.9,39.0
6,9905bb84-10f3-454e-b006-24fc3adba708,60.9,39.0
7,6881fd16-ba43-4689-b6bb-91a94ccf865a,50.6,44.5
8,30f09325-fbc3-4f5c-a478-50f50aad40fd,50.6,44.5
...,...,...,...
3241,75bfe37b-cf19-4c3e-bbf5-143955f9e640,45.1,35.1
3242,09efb9b4-b735-49ad-890b-0a7583496c00,40.7,31.1
3243,85737fb5-4776-4953-bb11-dccbe4d21ec7,40.7,31.1
3244,8f58481a-7d84-4db9-a4fa-97365676c2d2,41.6,31.1


## 5.2. pass_details
There a lot of field held with the pass subset which need unpacking. To start I am extracting the ID for which unpacking fields can be joined back into


In [51]:
passBase = pass_details[['id']]
passBase

,id
4,0613063a-1cd4-4a18-83a7-9722be2d9f40
6,9905bb84-10f3-454e-b006-24fc3adba708
9,32b8c47b-4410-4e2a-874c-c1711a74f758
12,df42110c-c91e-4b5a-92d9-21d0a15b9923
15,dfab4c23-01f7-4d3f-81e6-4c1c23205e9c
...,...
3232,06296dde-c59e-4328-8735-4ec3ed4b7b79
3234,eb6f3d10-5e0c-4dd3-923c-ced11b9b54d6
3238,fabe53c1-be33-434d-9523-c8f336f419d5
3240,b0cc0f48-5eaa-4f6f-8c34-3d6c139ad60a


### 5.2.1 Pass Cleaning Function
defining a function to clean up the fields which are made up of `id` and `name`

In [52]:
def pass_id_fields_clean(field_name):
  df = pass_details.query(f"{field_name} == {field_name}")[["id",f"{field_name}"]]
  # example: df = pass_details.query("recipient == recipient")[["id","recipient"]] 

  listNames = [f"{field_name}_id",f"{field_name}"]
  # example: listNames = ["recipient_id","recipient"]

  exec("df[listNames] = pd.DataFrame(df.{}.values.tolist(), index= df.index)".format(field_name))
  # example: df[listNames] = pd.DataFrame(df.field_name.values.tolist(), index= df.index)

  return df

In [53]:
pass_details_recipient = pass_id_fields_clean("recipient")
pass_details_height = pass_id_fields_clean("height")
pass_details_type_ = pass_id_fields_clean("type_")
pass_details_outcome = pass_id_fields_clean("outcome")
pass_details_technique = pass_id_fields_clean("technique") # this informaiton is also held with binary fields


###5.2.2 Pass end coordinates
Pass end coordinates need specific field names so can't be parsed using the above function

In [54]:
pass_details_end_location = pass_details.query("end_location == end_location")[["id","end_location"]]
listNames = ["x_end_location","y_end_location"]
pass_details_end_location[listNames] = pd.DataFrame(pass_details_end_location.end_location.values.tolist(), index= pass_details_end_location.index)
#pass_details_end_location['y_end_location'] = 80 - pass_details_end_location['y_end_location']
pass_details_end_location

,id,end_location,x_end_location,y_end_location
4,0613063a-1cd4-4a18-83a7-9722be2d9f40,"[60.9, 39.0]",60.9,39.0
6,9905bb84-10f3-454e-b006-24fc3adba708,"[50.6, 44.5]",50.6,44.5
9,32b8c47b-4410-4e2a-874c-c1711a74f758,"[40.0, 12.5]",40.0,12.5
12,df42110c-c91e-4b5a-92d9-21d0a15b9923,"[27.7, 28.7]",27.7,28.7
15,dfab4c23-01f7-4d3f-81e6-4c1c23205e9c,"[9.3, 37.2]",9.3,37.2
...,...,...,...,...
3232,06296dde-c59e-4328-8735-4ec3ed4b7b79,"[103.8, 38.1]",103.8,38.1
3234,eb6f3d10-5e0c-4dd3-923c-ced11b9b54d6,"[111.9, 50.8]",111.9,50.8
3238,fabe53c1-be33-434d-9523-c8f336f419d5,"[74.8, 45.0]",74.8,45.0
3240,b0cc0f48-5eaa-4f6f-8c34-3d6c139ad60a,"[79.4, 49.0]",79.4,49.0


###5.2.3. Numerical pass details: length and angle

In [55]:
pass_details_length = pass_details.query("length == length")[["id","length"]]
pass_details_angle = pass_details.query("angle == angle")[["id","angle"]]

### 5.2.4 Binary Pass Details
A lot of pass details are binary, i.e. true or false. These subsets don't need a function to clean instead the Null values where the field is not relevant can be removed

In [56]:
pass_details_switch = pass_details.query("switch == switch")[["id","switch"]]
pass_details_cross = pass_details.query("cross == cross")[["id","cross"]]
pass_details_assisted_shot_id = pass_details.query("assisted_shot_id == assisted_shot_id")[["id","assisted_shot_id"]]
pass_details_shot_assist = pass_details.query("shot_assist == shot_assist")[["id","shot_assist"]]
pass_details_aerial_won = pass_details.query("aerial_won == aerial_won")[["id","aerial_won"]]
pass_details_deflected = pass_details.query("deflected == deflected")[["id","deflected"]]
pass_details_inswinging = pass_details.query("inswinging == inswinging")[["id","inswinging"]]
pass_details_outswinging = pass_details.query("outswinging == outswinging")[["id","outswinging"]]
pass_details_goal_assist = pass_details.query("goal_assist == goal_assist")[["id","goal_assist"]]
pass_details_through_ball = pass_details.query("through_ball == through_ball")[["id","through_ball"]]
pass_details_no_touch = pass_details.query("no_touch == no_touch")[["id","no_touch"]]
pass_details_miscommunication = pass_details.query("miscommunication == miscommunication")[["id","miscommunication"]]
pass_details_straight = pass_details.query("straight == straight")[["id","straight"]]


###5.2.5. Creating Final Pass Subsets


In [57]:
# pass locations subset

#pass end location #length #angle #height
tempLocationDF1 = pass_details_end_location.merge(pass_details_length, on='id', how='left')
tempLocationDF2 = tempLocationDF1.merge(pass_details_angle, on='id', how='left')
tempLocationDF3 = tempLocationDF2.merge(pass_details_height, on='id', how='left')
pass_details_location = tempLocationDF3[['id','x_end_location','y_end_location','length','angle','height']]

In [ ]:
#binaries subset

tempBinaryDF1 = pass_details_switch.merge(pass_details_cross, on='id', how='outer')
tempBinaryDF2 = tempBinaryDF1.merge(pass_details_assisted_shot_id, on='id', how='outer')
tempBinaryDF3 = tempBinaryDF2.merge(pass_details_shot_assist, on='id', how='outer')
tempBinaryDF4 = tempBinaryDF3.merge(pass_details_aerial_won, on='id', how='outer')
tempBinaryDF5 = tempBinaryDF4.merge(pass_details_deflected, on='id', how='outer')
tempBinaryDF6 = tempBinaryDF5.merge(pass_details_inswinging, on='id', how='outer')
tempBinaryDF7 = tempBinaryDF6.merge(pass_details_outswinging, on='id', how='outer')
tempBinaryDF8 = tempBinaryDF7.merge(pass_details_goal_assist, on='id', how='outer')
tempBinaryDF9 = tempBinaryDF8.merge(pass_details_through_ball, on='id', how='outer')
tempBinaryDF10 = tempBinaryDF9.merge(pass_details_no_touch, on='id', how='outer')
tempBinaryDF11 = tempBinaryDF10.merge(pass_details_miscommunication, on='id', how='outer')
tempBinaryDF12 = tempBinaryDF11.merge(pass_details_straight, on='id', how='outer')
passBinaries = tempBinaryDF12

passMain is a subset comprising of names, types, outcomes and locations

In [59]:
tempPassDF1 = passBase.merge(pass_details_recipient, on='id', how='left')
tempPassDF2 = tempPassDF1.merge(pass_details_type_, on='id', how='left')
tempPassDF3 = tempPassDF2.merge(pass_details_outcome, on='id', how='left')
tempPassDF4 = tempPassDF3.merge(pass_details_location, on='id', how='left')

passMain = tempPassDF4.rename(columns={"recipient": "recipient_name",
                                       "type_": "pass_type_name",
                                       "type__id": "pass_type_id",
                                       "outcome": "pass_outcome_name",
                                       "outcome_id": "pass_outcome_id"})

In [ ]:
passMain

In [61]:
passBinaries  

,id,switch,cross,assisted_shot_id,shot_assist,aerial_won,deflected,inswinging,outswinging,goal_assist,through_ball,no_touch,miscommunication,straight
0,0be483f4-9463-49ba-b2ce-16fd5effa56f,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a9bcd214-6ee7-481a-a3ea-b090a2ef0b90,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93001c76-ec47-4fde-bb90-502d92974b96,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9691b9cd-cf0e-439b-add0-bc8c3e85e77a,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,413e44d2-b8b1-437f-98e3-ad7bacfd07c7,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3182,63047a29-b5e5-43da-bf5b-06f9bdf1e46e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3183,330dba94-9a9d-48bd-9f02-be4ae0b1eacd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3184,b7e0ab5e-631d-43a9-8e2d-761b20be8c43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3185,122235ae-f56b-4b45-b6e9-a41eed069d10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


##5.3. shot
the shot subset is much like the pass details subset in that is has several fields to unpack


In [62]:
# shot

In [152]:
shotBase = shot[['id','statsbomb_xg','key_pass_id']]

,id,statsbomb_xg,key_pass_id
158,095d8b43-5130-4849-b76e-fbd71dc4899a,0.012198,70e1fd39-5ea8-4fcc-86cc-f34ea431c118
338,d57626dc-08f0-4777-a05e-b6667744a8d3,0.011761,74875bb2-02c7-4e03-8464-1a4c4ee59c7d
376,e63b47a6-bc51-4dae-9a66-27c253121270,0.053545,3c269ea7-42a9-4e17-8f88-28b8331b6e03
829,e73cc30b-eba5-4177-870c-5fde2a3865c4,0.014514,NaN
1039,0d622785-15d0-42e7-ade2-4dd1f55da46a,0.152389,26678b20-21fa-4d86-aeb4-81e29469c755
...,...,...,...
2715,af29f86f-3a68-45a3-a491-4becf1840c5c,0.007020,024ab8dd-2892-452a-b2db-2231e6d34cf6
2739,461c0ca9-d45e-4cd3-9ee2-35a762790cef,0.129429,f1c4b255-5608-405b-b7cc-148a0788b597
2814,ba4b09d4-e8aa-4223-a337-5f2aa44ed16b,0.028548,f0257f68-8932-40cd-a04f-00b4da488ed0
2963,a552f030-e8b9-49c8-ae1b-8030f47869f2,0.035564,97be69d0-d605-47eb-bf5b-154cda03ee20


In [64]:
shotEndLocation = shot.query("end_location == end_location")[["id","end_location"]]
listNames = ["x_end_location","y_end_location","height"]
shotEndLocation[listNames] = pd.DataFrame(shotEndLocation.end_location.values.tolist(), index= shotEndLocation.index)
#shotEndLocation['y_end_location'] = 80 - shotEndLocation['y_end_location']
shotEndLocation

,id,end_location,x_end_location,y_end_location,height
158,095d8b43-5130-4849-b76e-fbd71dc4899a,"[120.0, 33.6, 5.0]",120.0,33.6,5.0
338,d57626dc-08f0-4777-a05e-b6667744a8d3,"[120.0, 30.1, 0.5]",120.0,30.1,0.5
376,e63b47a6-bc51-4dae-9a66-27c253121270,"[120.0, 31.6, 0.0]",120.0,31.6,0.0
829,e73cc30b-eba5-4177-870c-5fde2a3865c4,"[119.0, 40.0, 2.2]",119.0,40.0,2.2
1039,0d622785-15d0-42e7-ade2-4dd1f55da46a,"[120.0, 38.2, 1.6]",120.0,38.2,1.6
...,...,...,...,...,...
2715,af29f86f-3a68-45a3-a491-4becf1840c5c,"[103.8, 40.0]",103.8,40.0,NaN
2739,461c0ca9-d45e-4cd3-9ee2-35a762790cef,"[118.3, 38.0, 1.3]",118.3,38.0,1.3
2814,ba4b09d4-e8aa-4223-a337-5f2aa44ed16b,"[120.0, 45.4, 4.5]",120.0,45.4,4.5
2963,a552f030-e8b9-49c8-ae1b-8030f47869f2,"[120.0, 36.1, 5.3]",120.0,36.1,5.3


In [65]:
def shot_id_fields_clean(field_name):
  df = shot.query(f"{field_name} == {field_name}")[["id",f"{field_name}"]]
  listNames = [f"{field_name}_id",f"{field_name}"]
  exec("df[listNames] = pd.DataFrame(df.{}.values.tolist(), index= df.index)".format(field_name))
  return df

In [ ]:
shotType = shot_id_fields_clean("type_")
shotOutcome = shot_id_fields_clean("outcome")
shotTechnique = shot_id_fields_clean("technique")
shotBodyPart = shot_id_fields_clean("body_part")

In [67]:
tempShotsDF1 = shotBase.merge(shotEndLocation, on='id', how='left')
tempShotsDF2 = tempShotsDF1.merge(shotType, on='id', how='left')
tempShotsDF3 = tempShotsDF2.merge(shotOutcome, on='id', how='left')
tempShotsDF4 = tempShotsDF3.merge(shotTechnique, on='id', how='left')
tempShotsDF5 = tempShotsDF4.merge(shotBodyPart, on='id', how='left')

In [ ]:
shotMain_ = tempShotsDF5.rename(columns={"height": "shot_height",
                                        "type_": "shot_type_name",
                                        "type__id": "shot_type_id",
                                        "outcome": "shot_outcome_name",
                                        "outcome_id": "shot_outcome_id",
                                        "technique": "shot_technique_name",
                                        "technique_id": "shot_technique_id",
                                        "body_part": "shot_body_part_name",
                                        "body_part_id": "shot_body_part_id"})

shotMain = shotMain_.drop(columns=['end_location'])

In [ ]:
shotMain

##5.4. carry
the carry subsets only has one field to unpack but it is double nested

In [70]:
carry

tempCarry = carry.query("carry == carry")
listNames = ["end_location"]
tempCarry[listNames] = pd.DataFrame(tempCarry.carry.values.tolist(), index= tempCarry.index)

tempCarry2 = tempCarry.query("end_location == end_location")[["id","end_location"]]
listNames = ["x_end_location","y_end_location"]
tempCarry2[listNames] = pd.DataFrame(tempCarry2.end_location.values.tolist(), index= tempCarry2.index)
#tempCarry2['y_end_location'] = 80 - tempCarry2['y_end_location']
carryMain = tempCarry2[['id','x_end_location','y_end_location']]
carryMain

,id,x_end_location,y_end_location
8,30f09325-fbc3-4f5c-a478-50f50aad40fd,49.6,43.9
11,3fcf0768-a5b5-4bab-ac61-cb554ae17878,35.8,13.4
14,ffc20c99-60f9-4c51-96eb-67881322a584,27.7,28.9
21,2c91abe5-8a39-4211-b10a-bcb2cc824276,63.7,38.6
26,502e9ee7-60dd-4198-83a7-316efbae4204,70.4,53.3
...,...,...,...
3220,2f414d1b-9005-4edd-9938-6de6caac3893,35.8,21.6
3223,7d3386ec-04e8-4d2d-b516-cdf7cadb0262,48.2,26.6
3227,de00e0e4-5402-49b9-82bd-6188f7aa727f,77.8,43.6
3230,3238e1eb-c38e-4737-97aa-126fe60dfee3,84.2,55.3


##5.5. ball recovery
ball rec doesn't actually have any fields to unpack

In [71]:
ball_recovery

,id,recovery_failure,offensive
168,a2f36e88-264f-4fea-897f-93a07074f575,True,NaN
740,fe9581df-124d-413f-b07c-c02d8c199d7e,True,NaN
741,b1580dcb-32e6-4581-8bbf-9f1bbca26089,True,NaN
832,7df19aae-d58b-4b08-8b80-5de49dfc2151,True,NaN
988,c23fc682-1dfc-4392-b7d9-a5bf71783863,True,NaN
...,...,...,...
1972,225752c6-f1b1-47e8-98bd-389ddd98d85a,True,NaN
2140,e6763f16-441b-4ff6-95e9-79f8eba27666,True,NaN
2616,90423c62-a8d2-4520-b89b-f0bf7daf8c83,True,NaN
2825,d02ea60b-261a-45d5-b7e9-3c43656183f8,True,NaN


##5.6. duel


In [72]:
# duel

In [73]:
def duel_id_fields_clean(field_name):
  df = duel.query(f"{field_name} == {field_name}")[["id",f"{field_name}"]]
  listNames = [f"{field_name}_id",f"{field_name}"]
  exec("df[listNames] = pd.DataFrame(df.{}.values.tolist(), index= df.index)".format(field_name))
  return df

In [74]:
duelType = duel_id_fields_clean("type_")
duelOutcome = duel_id_fields_clean("outcome")

In [75]:
duelID = duel[['id']]

tempDuelDF1 = duelID.merge(duelType, on='id', how='left')
tempDuelDF2 = tempDuelDF1.merge(duelOutcome, on='id', how='left')

duelMain = tempDuelDF2.rename(columns={"type_": "duel_type_name",
                                       "type__id": "duel_type_id",
                                       "outcome": "outcome_type_name",
                                       "outcome_id": "outcome_type_id"})

In [76]:
duelMain

,id,duel_type_name,duel_type_id,outcome_type_name,outcome_type_id
0,7b3e4205-081a-4713-9f61-520606a4f0f4,Tackle,11,Success In Play,16.0
1,e40169c7-100a-4475-b951-41a8a8dc26cf,Tackle,11,Won,4.0
2,7377cc8d-95c4-437d-8337-c778886d1159,Tackle,11,Success In Play,16.0
3,bcf41c6d-025f-4c16-9b3c-528248f900e4,Tackle,11,Success In Play,16.0
4,ca6b1c2b-8969-423c-a4ec-8b47a2abf119,Tackle,11,Lost In Play,13.0
...,...,...,...,...,...
2885,b3ad36ed-41a2-4d5d-9b9b-a2dbdc5c152d,Aerial Lost,10,NaN,NaN
2886,4c9cfd12-39db-45d3-b3e2-093a7ec23abe,Aerial Lost,10,NaN,NaN
2887,50a090d4-7ed4-479f-b058-32a610b14534,Aerial Lost,10,NaN,NaN
2888,0e72364e-6aa4-48d0-87ec-6e7d70de28c1,Aerial Lost,10,NaN,NaN


##5.7. dribble

In [77]:
# dribble

In [78]:
def dribble_id_fields_clean(field_name):
  df = dribble.query(f"{field_name} == {field_name}")[["id",f"{field_name}"]]
  listNames = [f"{field_name}_id",f"{field_name}"]
  exec("df[listNames] = pd.DataFrame(df.{}.values.tolist(), index= df.index)".format(field_name))
  return df

In [79]:
dribbleID = dribble[['id','overrun','nutmeg','no_touch']]
dribbleOutcome = dribble_id_fields_clean("outcome")

dribbleMain_ = dribbleID.merge(dribbleOutcome, on='id', how='left')

dribbleMain = dribbleMain_.rename(columns={"outcome": "dribble_outcome_name",
                                             "outcome_id": "dribble_outcome_id"})

In [80]:
dribbleMain

,id,overrun,nutmeg,no_touch,dribble_outcome_name,dribble_outcome_id
0,4fce26ae-ea51-4a89-abcc-590b17f0a759,NaN,NaN,NaN,Incomplete,9
1,0550c27f-a304-4335-90a8-85e7a95f8218,NaN,NaN,NaN,Complete,8
2,0ae548b3-4375-498e-b9c6-8cb9e21d42ba,NaN,NaN,NaN,Incomplete,9
3,ec564a08-d965-4a15-bf87-b08ffaadb7bc,NaN,NaN,NaN,Complete,8
4,bb615328-a1c3-4076-ade1-32e0ba9c9c39,True,NaN,NaN,Incomplete,9
...,...,...,...,...,...,...
1393,822ea494-53a9-41a8-b927-c93c81659da8,NaN,NaN,NaN,Complete,8
1394,fc6f3b19-4097-41ec-a50e-8a06cff54150,NaN,NaN,NaN,Complete,8
1395,a3c81b81-35a3-4349-863a-98129db07a00,NaN,NaN,NaN,Complete,8
1396,f700f24a-a598-41f3-b7b5-d55fe38a3f4b,NaN,NaN,NaN,Incomplete,9


##5.8. interception

In [81]:
interception

tempInterception = interception.query("interception == interception")
listNames = ["outcome"]
tempInterception[listNames] = pd.DataFrame(tempInterception.interception.values.tolist(), index= tempInterception.index)

tempInterception2 = tempInterception.query("outcome == outcome")[["id","outcome"]]
listNames = ["outcome_id","outcome"]
tempInterception2[listNames] = pd.DataFrame(tempInterception2.outcome.values.tolist(), index= tempInterception2.index)
interceptionMain_ = tempInterception2[['id','outcome', 'outcome_id']]

interceptionMain = interceptionMain_.rename(columns={"outcome": "interception_outcome_name",
                                                     "outcome_id": "interception_outcome_id"})

interceptionMain

,id,interception_outcome_name,interception_outcome_id
590,dc2f2e7b-38b1-43ce-9c59-8daf9a9685d9,Won,4
732,ce54da01-a417-4252-9c27-1af5c92366f4,Success In Play,16
973,0fec7b4d-1e42-4213-ac3e-92fa5bab8393,Lost In Play,13
1032,b4282a2c-bb14-4aa1-8824-252ac1c68756,Won,4
1062,cd737615-62ca-4a83-9a29-fda90fef8ee8,Lost In Play,13
...,...,...,...
1059,7eb82c7e-afa6-4d33-acf2-e664ec70e76b,Success In Play,16
1068,6220843b-d9db-4b2f-8958-f2bac52585d6,Won,4
1236,9cecd82e-9815-44b8-857e-2909a1e8d71e,Lost In Play,13
2377,2c25c03d-94d1-41df-8f78-fb7527de9e0d,Won,4


##5.9. ball_receipt

In [82]:
ball_receipt

tempBallReceipt = ball_receipt.query("ball_receipt == ball_receipt")
listNames = ["ball_receiptOutcome"]
tempBallReceipt[listNames] = pd.DataFrame(tempBallReceipt.ball_receipt.values.tolist(), index= tempBallReceipt.index)

tempBallReceipt2 = tempBallReceipt.query("ball_receiptOutcome == ball_receiptOutcome")[["id","ball_receiptOutcome"]]
listNames = ["outcome_id","outcome"]
tempBallReceipt2[listNames] = pd.DataFrame(tempBallReceipt2.ball_receiptOutcome.values.tolist(), index= tempBallReceipt2.index)
# ballReceiptMain = tempBallReceipt2[['id','outcome']]

ballReceiptMain_ = tempBallReceipt2[['id','outcome', 'outcome_id']]

ballReceiptMain = ballReceiptMain_.rename(columns={"outcome": "ballReceipt_outcome_name",
                                                   "outcome_id": "ballReceiptn_outcome_id"})

ballReceiptMain

,id,ballReceipt_outcome_name,ballReceiptn_outcome_id
51,afa509bd-2e6d-4849-abbc-9ffe2fdc7082,Incomplete,9
101,2d84522c-1497-4ba1-84ae-37076ce81507,Incomplete,9
107,ac0aa36f-b74c-4377-ad2e-b4118455a5db,Incomplete,9
161,b32b8c36-7b65-4120-add1-b355c9800594,Incomplete,9
165,e51e0841-db54-42f3-8bdd-6d401ca99c4e,Incomplete,9
...,...,...,...
3163,ff55ff14-74cd-4059-ae3a-b281f5098eee,Incomplete,9
3172,cb3ae329-f7e4-4d41-894b-4e18e08db46c,Incomplete,9
3179,ae5e08b1-789c-498d-aee1-b2f8a9b34c6e,Incomplete,9
3187,09c8003f-a2fa-4284-b234-4003da32b051,Incomplete,9


#6. Developed Datasets
just a list of all the datasets that are ready to be pushed into a database

In [113]:
base_events

possession_team
play_pattern
team
player
position
counterpress
under_pressure
block
foul_won
out

location
passMain
passBinaries
shotMain
carryMain
ballRecoveryMain = ball_recovery
duelMain
dribbleMain
interceptionMain
ballReceiptMain

,id,ballReceipt_outcome_name,ballReceiptn_outcome_id
51,afa509bd-2e6d-4849-abbc-9ffe2fdc7082,Incomplete,9
101,2d84522c-1497-4ba1-84ae-37076ce81507,Incomplete,9
107,ac0aa36f-b74c-4377-ad2e-b4118455a5db,Incomplete,9
161,b32b8c36-7b65-4120-add1-b355c9800594,Incomplete,9
165,e51e0841-db54-42f3-8bdd-6d401ca99c4e,Incomplete,9
...,...,...,...
3163,ff55ff14-74cd-4059-ae3a-b281f5098eee,Incomplete,9
3172,cb3ae329-f7e4-4d41-894b-4e18e08db46c,Incomplete,9
3179,ae5e08b1-789c-498d-aee1-b2f8a9b34c6e,Incomplete,9
3187,09c8003f-a2fa-4284-b234-4003da32b051,Incomplete,9


#7. Setting Up Database
The database I am going to be using is sqlite. The packages for this database are part of the standard python libary so don't need to be installed. This database is a local rather than remote meaning the data will be stored in my google drive. 

In [114]:
import sqlite3
import sqlalchemy # this may not be needed

##7.1. Setting up database connection

In [115]:
con = sqlite3.connect("/content/gdrive/MyDrive/Colab Notebooks/invinciblesSeason/premSeason_03.sqlite")
# con.close()

## 7.2. submitting base events
Submitting base events strangely was the most difficult subset to handle. This is because it has `timestamp` which has a `datetime` data type which was not compatible with `sqlite` to over come this I had change the data type to a string and if I want pull out that field into dataframe at a later date I will need to reparse it back into `datetime` format

In [116]:
base_events.dtypes

id                        object
match_id                   int64
index_                     int64
period                     int64
timestamp         datetime64[ns]
minute                     int64
second                     int64
possession                 int64
duration                 float64
related_events            object
dtype: object

In [117]:
dict = {"id": str,
        "match_id": str,
        "index_": int,
        "period": int,
        "timestamp": str,
        "minute": int,
        "second": int,
        "possession": int,
        "duration": float,
        "related_events": str}

In [118]:
baseEvents_temp = base_events.astype(dict)
baseEvents_temp.dtypes

id                 object
match_id           object
index_              int64
period              int64
timestamp          object
minute              int64
second              int64
possession          int64
duration          float64
related_events     object
dtype: object

submitting to database

In [119]:
baseEvents_temp.to_sql(name = "baseEvents", con=con, if_exists='replace', index=False)

checking it is uploaded

In [120]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", con)
#pd.read_sql_query("DROP TABLE df", con)

,name
0,ballRecoveryMain
1,duelMain
2,dribbleMain
3,interceptionMain
4,ballReceiptMain
5,possession_team
6,play_pattern
7,player
8,team
9,position


checking it can be read back down into a df

In [121]:
df_baseEvent_test = pd.read_sql_query("SELECT * FROM baseEvents", con, parse_dates=['timestamp'])

In [122]:
df_baseEvent_test.dtypes

id                        object
match_id                  object
index_                     int64
period                     int64
timestamp         datetime64[ns]
minute                     int64
second                     int64
possession                 int64
duration                 float64
related_events            object
dtype: object

##7.3. submitting single field subsets
All single field subset can be submitted as they are

In [123]:
# possession_team.dtypes
# play_pattern.dtypes
# player.dtypes
# team.dtypes
# position.dtypes
# counterpress.dtypes
# under_pressure.dtypes
# block.dtypes
# foul_won.dtypes
# out.dtypes

In [124]:
possession_team.to_sql(name = "possession_team", con=con, if_exists='replace', index=False)
df_possession_team_test = pd.read_sql_query("SELECT * FROM possession_team", con)

In [125]:
play_pattern.to_sql(name = "play_pattern", con=con, if_exists='replace', index=False)
df_play_pattern_test = pd.read_sql_query("SELECT * FROM play_pattern", con)

In [126]:
player.to_sql(name = "player", con=con, if_exists='replace', index=False)
df_player_test = pd.read_sql_query("SELECT * FROM player", con)

In [127]:
team.to_sql(name = "team", con=con, if_exists='replace', index=False)
df_team_test = pd.read_sql_query("SELECT * FROM team", con)

In [128]:
position.to_sql(name = "position", con=con, if_exists='replace', index=False)
df_position_test = pd.read_sql_query("SELECT * FROM position", con)

In [129]:
counterpress.to_sql(name = "counterpress", con=con, if_exists='replace', index=False)
df_counterpress_test = pd.read_sql_query("SELECT * FROM counterpress", con)

In [130]:
under_pressure.to_sql(name = "under_pressure", con=con, if_exists='replace', index=False)
df_under_pressure_test = pd.read_sql_query("SELECT * FROM under_pressure", con)

In [131]:
block.to_sql(name = "block", con=con, if_exists='replace', index=False)
df_block_test = pd.read_sql_query("SELECT * FROM block", con)

In [132]:
foul_won.to_sql(name = "foul_won", con=con, if_exists='replace', index=False)
df_foul_won_test = pd.read_sql_query("SELECT * FROM foul_won", con)

In [133]:
out.to_sql(name = "out", con=con, if_exists='replace', index=False)
df_out_test = pd.read_sql_query("SELECT * FROM out", con)

In [134]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", con)

,name
0,ballRecoveryMain
1,duelMain
2,dribbleMain
3,interceptionMain
4,ballReceiptMain
5,location
6,passMain
7,passBinaries
8,shotMain
9,carryMain


##7.4. submitting location
location can be submitted as is

In [135]:
location.dtypes

id     object
x     float64
y     float64
dtype: object

In [136]:
location.to_sql(name = "location", con=con, if_exists='replace', index=False)
df_location_test = pd.read_sql_query("SELECT * FROM location", con)
df_location_test.dtypes

id     object
x     float64
y     float64
dtype: object

##7.5. submitting pass subsets
both passMain and passBinaries can be submitted as is

###7.5.1. submitting passMain

In [137]:
passMain.dtypes
passMain.to_sql(name = "passMain", con=con, if_exists='replace', index=False)
df_passMain_test = pd.read_sql_query("SELECT * FROM passMain", con)
df_passMain_test.dtypes

id                    object
recipient_name        object
recipient_id         float64
pass_type_name        object
pass_type_id         float64
pass_outcome_name     object
pass_outcome_id      float64
x_end_location       float64
y_end_location       float64
length               float64
angle                float64
height                object
dtype: object

###7.5.2 submitting passBinaries

In [138]:
passBinaries.dtypes
passBinaries.to_sql(name = "passBinaries", con=con, if_exists='replace', index=False)
df_passBinaries_test = pd.read_sql_query("SELECT * FROM passBinaries", con)
passBinaries.dtypes

id                  object
switch              object
cross               object
assisted_shot_id    object
shot_assist         object
aerial_won          object
deflected           object
inswinging          object
outswinging         object
goal_assist         object
through_ball        object
no_touch            object
miscommunication    object
straight            object
dtype: object

##7.6. submitting shotMain
shotMain can be submitted as is

In [139]:
shotMain.dtypes
shotMain.to_sql(name = "shotMain", con=con, if_exists='replace', index=False)
df_shotMain_test = pd.read_sql_query("SELECT * FROM shotMain", con)
df_shotMain_test.dtypes

id                      object
statsbomb_xg           float64
key_pass_id             object
x_end_location         float64
y_end_location         float64
shot_height            float64
shot_type_name          object
shot_type_id             int64
shot_outcome_name       object
shot_outcome_id          int64
shot_technique_name     object
shot_technique_id        int64
shot_body_part_name     object
shot_body_part_id        int64
dtype: object

##7.7. submitting carryMain
can be submitted as is

In [140]:
carryMain.dtypes
carryMain.to_sql(name = "carryMain", con=con, if_exists='replace', index=False)
df_carryMain_test = pd.read_sql_query("SELECT * FROM carryMain", con)
df_carryMain_test.dtypes

id                 object
x_end_location    float64
y_end_location    float64
dtype: object

##7.8. submitting ballRecoveryMain
can be submitted as is

In [141]:
ballRecoveryMain.dtypes
ballRecoveryMain.to_sql(name = "ballRecoveryMain", con=con, if_exists='replace', index=False)
df_ballRecoveryMain_test = pd.read_sql_query("SELECT * FROM ballRecoveryMain", con)
df_ballRecoveryMain_test.dtypes

id                   object
recovery_failure    float64
offensive           float64
dtype: object

##7.9 submitting duelMain
can be submitted as is

In [142]:
duelMain.dtypes
duelMain.to_sql(name = "duelMain", con=con, if_exists='replace', index=False)
df_duelMain_test = pd.read_sql_query("SELECT * FROM duelMain", con)
df_duelMain_test.dtypes

id                    object
duel_type_name        object
duel_type_id           int64
outcome_type_name     object
outcome_type_id      float64
dtype: object

##7.10 submitting dribbleMain
can be submitted as is

In [143]:
dribbleMain.dtypes
dribbleMain.to_sql(name = "dribbleMain", con=con, if_exists='replace', index=False)
df_dribbleMain_test = pd.read_sql_query("SELECT * FROM dribbleMain", con)
df_dribbleMain_test.dtypes

id                       object
overrun                 float64
nutmeg                  float64
no_touch                float64
dribble_outcome_name     object
dribble_outcome_id        int64
dtype: object

##7.11 submitting interceptionMain
can be submitted as is

In [144]:
interceptionMain.dtypes
interceptionMain.to_sql(name = "interceptionMain", con=con, if_exists='replace', index=False)
df_interceptionMain_test = pd.read_sql_query("SELECT * FROM interceptionMain", con)
df_interceptionMain_test.dtypes

id                           object
interception_outcome_name    object
interception_outcome_id       int64
dtype: object

##7.12 submitting ballReceiptMain
can be submitted as is


In [145]:
ballReceiptMain.dtypes
ballReceiptMain.to_sql(name = "ballReceiptMain", con=con, if_exists='replace', index=False)
df_ballReceiptMain_test = pd.read_sql_query("SELECT * FROM ballReceiptMain", con)
df_ballReceiptMain_test.dtypes

id                          object
ballReceipt_outcome_name    object
ballReceiptn_outcome_id      int64
dtype: object

#8. Checking Database

##8.1. Tables held in the database

In [ ]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", con)

##8.2. test query

In [ ]:
sql = '''
SELECT *
FROM baseEvents b
LEFT JOIN possession_team USING(id)
LEFT JOIN team USING(id)
LEFT JOIN play_pattern USING(id)
LEFT JOIN player USING(id)
LEFT JOIN position USING(id)
LEFT JOIN location using(id)
'''

dfTest1 = pd.read_sql_query(sql, con)

#9. Close database connection

In [148]:
# con.close()

# Next Developments/Improvments

- dimensions should join by category id rather than event id and id_key should be held in fact table to start. For example, the dataset `player` should be a dimension table which just holds all the player names and ID's and joined to a player ID held within `base_events` that acts at the fact table

- pass binaries should be broken up and be joined using a dimension method as above

- null values should be cleaned up and replaces with informations, i.e. completes for pass outcome for true for nulls

- remove accented charactsters from names. this will be easier if dimension tables are created for names

- submit `competition` and `match` subset into the database 

- put together documentation to explain columns in table

In [ ]:
player 